In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [27]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_column', None)
df_engagement = pd.read_csv('../data/engagement_clusters.csv', na_values=['?', None, 'undefined'])
df_engagement.sample(5)

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,cluster
43533,33664755403,1,24200.0,756668358.0,0
49976,33667466157,1,102551.0,597798827.0,0
6946,33618885648,1,86400.0,238852745.0,1
43706,33664817276,1,86399.0,730079409.0,0
37533,33662644108,1,94287.0,634232178.0,0


In [28]:
df_experience = pd.read_csv('../data/experience_clusters.csv', na_values=['?', None, 'undefined'])
df_experience.sample(5)

,MSISDN/Number,Total RTT,Total TCP retransmission,Total Throughput,Handset Type,encoded,cluster
32362,33763734494,797.0,3462231.0,5029.0,Apple iPhone Se (A1723),57,0
10654,33659416397,1103.0,11289444.0,123959.0,Huawei B528S-23A,218,2
14875,33662229287,203.0,2423533.0,82204.0,Huawei B528S-23A,218,0
10402,33659288467,61.0,94783.0,40199.0,Apple iPhone 7 (A1778),46,0
25794,33684542746,55.0,50984.0,3631.0,Apple iPhone 6 (A1586),36,0


In [29]:
# how many missing values exist or better still what is the % of missing values in the dataset?
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

percent_missing(df_engagement)
percent_missing(df_experience)

The dataset contains 0.0 % missing values.
The dataset contains 0.0 % missing values.


In [30]:
less_engaged_centroid = df_engagement[df_engagement['cluster'] == 0].groupby('cluster').mean()
less_engaged_centroid

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data
cluster,,,,
0,4.234635e+10,1.0,101469.777192,7.064548e+08


In [31]:
def compute_engagement_score(df,centroid):
    x = float(centroid['Bearer Id'])
    y = float(centroid['Dur. (ms)'])
    z = float(centroid['Total Data'])
    df['engagement score'] = ((df['Bearer Id'] - x)**2 + (df['Dur. (ms)'] - y)**2 + (df['Total Data'] - z)**2)**0.5
    return df
df_engagement = compute_engagement_score(df_engagement,less_engaged_centroid)

In [32]:
df_engagement.head()

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,cluster,engagement score
0,33601001722,1,116720.0,878690574.0,0,1.722358e+08
1,33601001754,1,181230.0,156859643.0,1,5.495951e+08
2,33601002511,1,134969.0,595966483.0,0,1.104883e+08
3,33601007832,1,49878.0,422320698.0,1,2.841341e+08
4,33601011634,2,128360.0,654723066.0,2,5.173172e+07


In [33]:
worst_experience_centroid = df_experience[df_experience['cluster'] == 0]\
    [['Total RTT', 'Total TCP retransmission', 'Total Throughput', 'encoded', 'cluster']].groupby('cluster').mean()
worst_experience_centroid.head()

,Total RTT,Total TCP retransmission,Total Throughput,encoded
cluster,,,,
0,124.876095,4.584649e+06,42207.289489,145.086606


In [34]:
def compute_experience_score(df,centroid):
    x = float(centroid['Total RTT'])
    y = float(centroid['Total TCP retransmission'])
    z = float(centroid['Total Throughput'])
    w = float(centroid['encoded'])
    df['experience score'] = ((df['Total RTT'] - x)**2 + (df['Total TCP retransmission'] - y)**2 \
                              + (df['Total Throughput'] - z)**2 + (df['encoded'] - w)**2)**0.5
    return df
df_experience = compute_experience_score(df_experience,worst_experience_centroid)
df_experience.head()

,MSISDN/Number,Total RTT,Total TCP retransmission,Total Throughput,Handset Type,encoded,cluster,experience score
0,33601008617,91.0,9370832.0,56781.0,Apple iPhone Se (A1723),57,0,4.786205e+06
1,33601011634,39.0,110232.0,42416.0,Huawei Mate 10 Pro Porsche Design Huawei Mate 10,284,0,4.474417e+06
2,33601021217,160.0,14594645.0,19256.0,Apple iPhone 7 Plus (A1784),49,0,1.001002e+07
3,33601031129,60.0,2325497.0,38190.0,Apple iPhone 8 Plus (A1897),55,0,2.259155e+06
4,33601034530,656.0,2006261.0,8539.0,Apple iPhone 7 (A1778),46,0,2.578608e+06


In [35]:
df_satisfaction = pd.merge(df_engagement[['MSISDN/Number', 'engagement score']], df_experience[['MSISDN/Number', 'experience score']], on="MSISDN/Number")
df_satisfaction.head()

,MSISDN/Number,engagement score,experience score
0,33601011634,5.173172e+07,4.474417e+06
1,33601021217,7.636237e+07,1.001002e+07
2,33601031129,6.940172e+07,2.259155e+06
3,33601062786,6.326152e+07,2.747921e+06
4,33601071144,5.911604e+08,4.553683e+06


In [36]:
df_satisfaction['Satisfaction Score'] = (df_satisfaction['engagement score'] + df_satisfaction['experience score'])/2

In [37]:
df_satisfaction.head()

,MSISDN/Number,engagement score,experience score,Satisfaction Score
0,33601011634,5.173172e+07,4.474417e+06,2.810307e+07
1,33601021217,7.636237e+07,1.001002e+07,4.318620e+07
2,33601031129,6.940172e+07,2.259155e+06,3.583044e+07
3,33601062786,6.326152e+07,2.747921e+06,3.300472e+07
4,33601071144,5.911604e+08,4.553683e+06,2.978570e+08


### Top 10 satisfied customers

In [38]:
df_satisfaction[['MSISDN/Number', 'Satisfaction Score']].sort_values(by="Satisfaction Score", ascending=False).head(10)

,MSISDN/Number,Satisfaction Score
18829,33761164622,3.506217e+08
14162,33668534621,3.501762e+08
12565,33666235447,3.493557e+08
6295,33659111802,3.457023e+08
3251,33641356514,3.449822e+08
5924,33658792462,3.441726e+08
13649,33667821744,3.424084e+08
5598,33658511672,3.411603e+08
6752,33659472904,3.404411e+08
17009,33698481232,3.397354e+08


### Regression model for Satisfaction Score

In [39]:
df_feature = pd.merge(df_engagement[['MSISDN/Number', 'Bearer Id' , 'Dur. (ms)', 'Total Data']], \
                      df_experience[['MSISDN/Number', 'Total RTT', 'Total TCP retransmission', 'Total Throughput', 'encoded']], \
                      on="MSISDN/Number")
df_feature.head()

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,Total RTT,Total TCP retransmission,Total Throughput,encoded
0,33601011634,2,128360.0,654723066.0,39.0,110232.0,42416.0,284
1,33601021217,1,38416.0,630092434.0,160.0,14594645.0,19256.0,49
2,33601031129,1,55730.0,637053075.0,60.0,2325497.0,38190.0,55
3,33601062786,1,10532.0,769716233.0,55.0,1836731.0,46431.0,679
4,33601071144,1,86399.0,115294379.0,33.0,31013.0,62865.0,43


In [40]:
df_feature = pd.merge(df_feature, \
                      df_satisfaction[['MSISDN/Number', 'Satisfaction Score']], \
                      on="MSISDN/Number")
df_feature.head()

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,Total RTT,Total TCP retransmission,Total Throughput,encoded,Satisfaction Score
0,33601011634,2,128360.0,654723066.0,39.0,110232.0,42416.0,284,2.810307e+07
1,33601021217,1,38416.0,630092434.0,160.0,14594645.0,19256.0,49,4.318620e+07
2,33601031129,1,55730.0,637053075.0,60.0,2325497.0,38190.0,55,3.583044e+07
3,33601062786,1,10532.0,769716233.0,55.0,1836731.0,46431.0,679,3.300472e+07
4,33601071144,1,86399.0,115294379.0,33.0,31013.0,62865.0,43,2.978570e+08


In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
X = df_feature.loc[:, ['Bearer Id', 'Dur. (ms)', 'Total Data', 'Total RTT', 'Total TCP retransmission', 'Total Throughput', 'encoded']].values
X = StandardScaler().fit_transform(X)
X.shape

(21370, 7)

In [42]:
y =  df_feature.loc[:, ['Satisfaction Score']].values
y = StandardScaler().fit_transform(y)
y.shape

(21370, 1)

In [43]:
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.6745208819037907

In [44]:
#save model
pickle.dump(reg, open('../models/satisfaction_model.sav', 'wb'))

## K means on the engagement and experience scores

In [45]:
from sklearn.preprocessing import StandardScaler
x_score = df_satisfaction[['engagement score', 'experience score']].loc[:, :].values
x_score = StandardScaler().fit_transform(x_score)
x_score.shape

(21370, 2)

In [46]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(x_score)
df_satisfaction['cluster'] = kmeans.labels_

In [47]:
df_satisfaction.head()

,MSISDN/Number,engagement score,experience score,Satisfaction Score,cluster
0,33601011634,5.173172e+07,4.474417e+06,2.810307e+07,0
1,33601021217,7.636237e+07,1.001002e+07,4.318620e+07,0
2,33601031129,6.940172e+07,2.259155e+06,3.583044e+07,0
3,33601062786,6.326152e+07,2.747921e+06,3.300472e+07,0
4,33601071144,5.911604e+08,4.553683e+06,2.978570e+08,1


### Average engagement  and experience scores per cluster

In [51]:
df_satisfaction[['engagement score', 'experience score', 'cluster']].groupby('cluster').mean()

,engagement score,experience score
cluster,,
0,1.315389e+08,6.513352e+06
1,4.575508e+08,6.379523e+06


In [52]:
df_satisfaction.to_csv('../data/user_experience_scores.csv', index=False)